# Этап 1. Тест-запрос на сервер и ответ

На данном этапе с помощью библиотеки requests осуществляется тестовый запрос на сайт и выводится ответ с сервера.

In [ ]:
!python3 -m pip install requests

In [106]:
import requests
import json
import datetime
url="ссылка на сайт"
null=None

In [107]:
print(str(datetime.datetime.now()))

2022-12-22 12:08:48.809329


In [128]:
mydata=[
{
    "id": "string",
    "pickupInterval": {
      "start": str(datetime.datetime.now()),
      "finish": str(datetime.datetime.now())
    }, 
    "dropoffInterval": {
      "start": str(datetime.datetime.now()),
      "finish": str(datetime.datetime.now())
    },
    "pickupLocation": {
      "id": "string",
      "name": "Омск (Омская область)",
      "latitude": 54.9848136,
      "longitude": 73.3674638
    },
    "dropoffLocation": {
      "id": "string",
      "name": "Самара (Самарская область)",
      "latitude": 54.9848136,
      "longitude": 73.3674638
    },
    "packs": [
      {
        "id": "string",
        "pack": {
          "id": "string",
          "items": [
            {
              "id": "string",
              "item": {
                "id": "string",
                "name": "трубы",
                "length": 750,
                "width": 125,
                "height": 150,
                "weight": 20000,
                "unit": "string",
                "cost": 0,
                "type": null
              },
              "quantity": 1
            }
          ],
          "name": "трубы",
          "length": 750,
          "width": 125,
          "height": 150,
          "weight": 20000,
          "type": null
        },
        "quantity": 1
      }
    ]
  }
]

In [129]:
r = requests.post(url, json = mydata)
print(r.text)

{"id":null,"costs":[],"description":"Заказ0: маршрутов нет"}


# Этап 2. Работа с данными excel

Подгружаем данные из excel, обрабатываем их, создаем пустые колонки для записи новых данных, которые будем получать по API. </br>

In [51]:
import requests
import pandas as pd
import numpy as np
import json

In [87]:
file = 'test_daily_report.xlsx'

In [88]:
data_tab = pd.read_excel(file, sheet_name='Для теста')

In [95]:
data_tab['cost1']=np.NaN
data_tab['date1']=np.NaN
data_tab['route_length1']=np.NaN
data_tab['load_capacity1']=np.NaN
data_tab['description1']=np.NaN
data_tab['cost2']=np.NaN
data_tab['date2']=np.NaN
data_tab['route_length2']=np.NaN
data_tab['load_capacity2']=np.NaN
data_tab['description2']=np.NaN
data_tab['cost3']=np.NaN
data_tab['date3']=np.NaN
data_tab['route_length3']=np.NaN
data_tab['load_capacity3']=np.NaN
data_tab['description3']=np.NaN
data_tab['cost4']=np.NaN
data_tab['date4']=np.NaN
data_tab['route_length4']=np.NaN
data_tab['load_capacity4']=np.NaN
data_tab['description4']=np.NaN
data_tab['deliveryCompany']=np.NaN
data_tab['description']=np.NaN

In [96]:
data_tab.head()

,ID,Статус,Дата создания заказа,Дата загрузки,Категория ТС,Тип кузова,Груз,Упаковка,Город отгрузки,Город выгрузки,...,cost4,date4,route_length4,load_capacity4,deliveryCompany,description,description1,description2,description3,description4
0,9620,Подтверждён,44872,44873,20 т / 82 м3,Тентованный,Смазочные материалы,Паллета EUR,"Россия, Московская область, Фрязино, Озёрная у...","Краснодарский край, Новороссийск, село Цемдоли...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,11141,Подтверждён,44888,44894,20 т / 82 м3,Тентованный,Смазочные материалы,Паллета EUR,"Ярославская область, Ярославль",Москва,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4658,Подтверждён,44797,44801,20 т / 82 м3,Тентованный,Смазочные материалы,Паллета EUR,"Омская область, Омск","Причальная улица, 11/1, Томск",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9954,Подтверждён,44875,44877,20 т / 82 м3,Тентованный,Смазочные материалы,Паллета EUR,"проспект Губкина, 1, Омск","Техническая улица, 15, Ленинский район, Киров",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11818,Подтверждён,44894,44896,20 т / 82 м3,Трал,Вагон-дом,Тарные места,"Россия, Тюменская область,г. Тюмень,ул. Мелиор...","Россия, Иркутская область, Усть-Кут, Луговая у...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Этап 3. Создание функции для запроса по API сайта

Создаем функцию, которая будет формировать запрос на сервер с информацией о грузе, состоящем из одного предмета.

In [123]:
def onestr(i, data_table):
    ad_fr = data_table['ad_from'][i]
    lat_fr = data_table['latitude_from'][i]
    long_fr = data_table['longitude_from'][i]
    ad_t = data_table['ad_to'][i]
    lat_t = data_table['latitude_to'][i]
    long_t = data_table['longitude_to'][i]
    leng = data_table['length'][i]
    wid = data_table['width'][i]
    hei = data_table['height'][i]
    weig = data_table['weight'][i]
    quant = data_table['quantity'][i]
    jsonfile=[
          {
            "id": "string",
            "pickupInterval": {
              "start": null,
              "finish": null
            },
            "dropoffInterval": {
              "start": null,
              "finish": null
            },
            "pickupLocation": {
              "id": "string",
              "name": ad_fr,
              "latitude": lat_fr,
              "longitude": long_fr
            },
            "dropoffLocation": {
              "id": "string",
              "name": ad_t,
              "latitude": lat_t,
              "longitude": long_t
            },
            "packs": [
              {
                "id": "string",
                "pack": {
                  "id": "string",
                  "items": [
                    {
                      "id": "string",
                      "item": {
                        "id": "string",
                        "name": "string",
                        "length": float(leng),
                        "width": float(wid),
                        "height": float(hei),
                        "weight": float(weig),
                        "unit": "string",
                        "cost": 0,
                        "type": null
                      },
                      "quantity": 1
                    }
                  ],
                  "name": "string",
                  "length": float(leng),
                  "width": float(wid),
                  "height": float(hei),
                  "weight": float(weig),
                  "type": null
                },
                "quantity": int(quant)
              }
            ]
          }
        ]
    print(jsonfile)
    req = requests.post("ссылка на сайт", json = jsonfile)
    newdata = req.text
    comm = json.loads(newdata)['description']
    data_table.loc[i, ['description']] = comm
    reqtext = json.loads(newdata)['costs']
    # Записываем полученные данные в колонки таблицы в зависимости от того, сколько данных получено
    if len(reqtext)>=1:
        cost1 = dict(reqtext[0])['routeCost']
        date1 = dict(reqtext[0])['deliveryTime']
        routelength1=dict(reqtext[0])['routeLength']
        load_cap1=dict(reqtext[0])['vechicleWeightCapacity']
        descr1=dict(reqtext[0])['description']
        data_table.loc[i, ['cost1']] = cost1
        data_table.loc[i, ['date1']] = date1
        data_table.loc[i, ['route_length1']] = routelength1
        data_table.loc[i, ['load_capacity1']] = load_cap1
        data_table.loc[i, ['description1']] = descr1
        if len(reqtext)==1:
            del_comp=dict(reqtext[0])['deliveryCompany']
            if dict(reqtext[0])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp
        if len(reqtext)>=2: 
            cost2 = dict(reqtext[1])['routeCost']
            date2 = dict(reqtext[1])['deliveryTime']
            routelength2=dict(reqtext[1])['routeLength']
            load_cap2=dict(reqtext[1])['vechicleWeightCapacity']
            del_comp=dict(reqtext[1])['deliveryCompany']
            descr2=dict(reqtext[1])['description']
            data_table.loc[i, ['cost2']] = cost2
            data_table.loc[i, ['date2']] = date2
            data_table.loc[i, ['route_length2']] = routelength2       
            data_table.loc[i, ['load_capacity2']] = load_cap2
            data_table.loc[i, ['description2']] = descr2
            if dict(reqtext[1])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp
        if len(reqtext)>=3: 
            cost3 = dict(reqtext[2])['routeCost']
            date3 = dict(reqtext[2])['deliveryTime']
            routelength3=dict(reqtext[2])['routeLength']
            load_cap3=dict(reqtext[2])['vechicleWeightCapacity']
            descr3=dict(reqtext[2])['description']
            del_comp=dict(reqtext[2])['deliveryCompany']
            data_table.loc[i, ['cost3']] = cost3
            data_table.loc[i, ['date3']] = date3
            data_table.loc[i, ['route_length3']] = routelength3   
            data_table.loc[i, ['load_capacity3']] = load_cap3
            data_table.loc[i, ['description3']] = descr3
            if dict(reqtext[2])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp
        if len(reqtext)>=4: 
            cost4 = dict(reqtext[3])['routeCost']
            date4 = dict(reqtext[3])['deliveryTime']
            routelength4=dict(reqtext[3])['routeLength']
            load_cap4=dict(reqtext[3])['vechicleWeightCapacity']
            descr4=dict(reqtext[3])['description']
            del_comp=dict(reqtext[3])['deliveryCompany']
            data_table.loc[i, ['cost4']] = cost4
            data_table.loc[i, ['date4']] = date4
            data_table.loc[i, ['route_length4']] = routelength4    
            data_table.loc[i, ['load_capacity4']] = load_cap4
            data_table.loc[i, ['description4']] = descr4
            data_table.loc[i, ['deliveryCompany']] = del_comp
            if dict(reqtext[3])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp

Создаем функцию, которая будет формировать запрос на сервер с информацией о грузе, состоящем из нескольких предметов.

In [119]:
def fewstr(i, amount_str, data_table):
    ad_fr = data_table['ad_from'][i]
    lat_fr = data_table['latitude_from'][i]
    long_fr = data_table['longitude_from'][i]
    ad_t = data_table['ad_to'][i]
    lat_t = data_table['latitude_to'][i]
    long_t = data_table['longitude_to'][i]
    unite_text=[]
    leng=[]
    wid=[]
    hei=[]
    weig=[]
    quant=[]
    for j in range(amount_str-1):
        leng.append(data_table['length'][i+j+1])
        wid.append(data_table['width'][i+j+1])
        hei.append(data_table['height'][i+j+1])
        weig.append(data_table['weight'][i+j+1])
        quant.append(data_table['quantity'][i+j+1])
        insert_text={
                "id": "string",
                "pack": {
                  "id": "string",
                  "items": [
                    {
                      "id": "string",
                      "item": {
                        "id": "string",
                        "name": "string",
                        "length": float(leng[j]),
                        "width": float(wid[j]),
                        "height": float(hei[j]),
                        "weight": float(weig[j]),
                        "unit": "string",
                        "cost": 0,
                        "type": null
                      },
                      "quantity": 1
                    }
                  ],
                  "name": "string",
                  "length": float(leng[j]),
                  "width": float(wid[j]),
                  "height": float(hei[j]),
                  "weight": float(weig[j]),
                  "type": null
                },
                "quantity": int(quant[j])
              }
        unite_text.append(insert_text)
    jsonfile=[
          {
            "id": "string",
            "pickupInterval": {
              "start": "2022-12-22T07:00:37.786Z",
              "finish": "2022-12-22T07:00:37.786Z"
            },
            "dropoffInterval": {
              "start": "2022-12-22T07:00:37.786Z",
              "finish": "2022-12-22T07:00:37.786Z"
            },
            "pickupLocation": {
              "id": "string",
              "name": ad_fr,
              "latitude": lat_fr,
              "longitude": long_fr
            },
            "dropoffLocation": {
              "id": "string",
              "name": ad_t,
              "latitude": lat_t,
              "longitude": long_t
            },
            "packs": 
                unite_text
            
          }
    ]
    req = requests.post("ссылка на сайт", json = jsonfile)
    newdata = req.text
    comm = json.loads(newdata)['description']
    data_table.loc[i, ['description']] = comm
    reqtext = json.loads(newdata)['costs']
    # Записываем полученные данные в колонки таблицы в зависимости от того, сколько данных получено
    if len(reqtext)>=1:
        cost1 = dict(reqtext[0])['routeCost']
        date1 = dict(reqtext[0])['deliveryTime']
        routelength1=dict(reqtext[0])['routeLength']
        load_cap1=dict(reqtext[0])['vechicleWeightCapacity']
        data_table.loc[i, ['cost1']] = cost1
        data_table.loc[i, ['date1']] = date1
        data_table.loc[i, ['route_length1']] = routelength1
        data_table.loc[i, ['load_capacity1']] = load_cap1
        if len(reqtext)==1:
            del_comp=dict(reqtext[0])['deliveryCompany']
            data_table.loc[i, ['deliveryCompany']] = del_comp
        if len(reqtext)>=2: 
            cost2 = dict(reqtext[1])['routeCost']
            date2 = dict(reqtext[1])['deliveryTime']
            routelength2=dict(reqtext[1])['routeLength']
            load_cap2=dict(reqtext[1])['vechicleWeightCapacity']
            del_comp=dict(reqtext[1])['deliveryCompany']
            data_table.loc[i, ['cost2']] = cost2
            data_table.loc[i, ['date2']] = date2
            data_table.loc[i, ['route_length2']] = routelength2       
            data_table.loc[i, ['load_capacity2']] = load_cap2
            if dict(reqtext[1])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp
        if len(reqtext)>=3: 
            cost3 = dict(reqtext[2])['routeCost']
            date3 = dict(reqtext[2])['deliveryTime']
            routelength3=dict(reqtext[2])['routeLength']
            load_cap3=dict(reqtext[2])['vechicleWeightCapacity']
            del_comp=dict(reqtext[2])['deliveryCompany']
            data_table.loc[i, ['cost3']] = cost3
            data_table.loc[i, ['date3']] = date3
            data_table.loc[i, ['route_length3']] = routelength3   
            data_table.loc[i, ['load_capacity3']] = load_cap3
            if dict(reqtext[2])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp
        if len(reqtext)>=4: 
            cost4 = dict(reqtext[3])['routeCost']
            date4 = dict(reqtext[3])['deliveryTime']
            routelength4=dict(reqtext[3])['routeLength']
            load_cap4=dict(reqtext[3])['vechicleWeightCapacity']
            del_comp=dict(reqtext[3])['deliveryCompany']
            data_table.loc[i, ['cost4']] = cost4
            data_table.loc[i, ['date4']] = date4
            data_table.loc[i, ['route_length4']] = routelength4    
            data_table.loc[i, ['load_capacity4']] = load_cap4
            data_table.loc[i, ['deliveryCompany']] = del_comp
            if dict(reqtext[3])['description']=='Сборный (курьерской службой)':
                data_table.loc[i, ['deliveryCompany']] = del_comp

In [124]:
null=None
for k in range(len(id_values)):
    curr_data = data_tab[data_tab['ID']==id_values[k]]
    print(curr_data)
    # Для груза с одним предметом используем первую функцию onestr
    if curr_data.shape[0]==1:
        io=data_tab[data_tab['ID']==id_values[k]].index
        print(io[0])
        onestr(io[0], data_tab)
    # Для груза с несколькими предметами используем вторую функцию fewstr
    elif curr_data.shape[0]>1:
        io=data_tab[data_tab['ID']==id_values[k]].index
        print(io[0])
        fewstr(io[0], curr_data.shape[0], data_tab)

      ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
60  4753  Подтверждён                 44798          44802  20 т / 82 м3   

     Тип кузова                 Груз     Упаковка  \
60  Тентованный  Смазочные материалы  Паллета EUR   

                   Город отгрузки  Город выгрузки   ...  cost4  date4  \
60  Ярославская область, Ярославль            Омск  ...    NaN    NaN   

    route_length4  load_capacity4 deliveryCompany description   description1  \
60            NaN             NaN             NaN  Заказ0: ок  Отдельный А/М   

            description2  description3 description4  
60  ТЕСТ: Отдельный А/М.           NaN          NaN  

[1 rows x 52 columns]
60
[{'id': 'string', 'pickupInterval': {'start': None, 'finish': None}, 'dropoffInterval': {'start': None, 'finish': None}, 'pickupLocation': {'id': 'string', 'name': 'Ярославль (Ярославская область)', 'latitude': 57.6216145, 'longitude': 39.897878}, 'dropoffLocation': {'id': 'string', 'name': 'Омск 

       ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
66  11575  Подтверждён                 44890          44894  20 т / 82 м3   

     Тип кузова                 Груз     Упаковка Город отгрузки   \
66  Тентованный  Смазочные материалы  Паллета EUR       Ярославль   

                                      Город выгрузки   ...  cost4  date4  \
66  проезд Введенского, 12, Фрязино, Московская об...  ...    NaN    NaN   

    route_length4  load_capacity4 deliveryCompany description  \
66            NaN             NaN             NaN  Заказ0: ок   

            description1  description2  description3 description4  
66  ТЕСТ: Отдельный А/М.           NaN           NaN          NaN  

[1 rows x 52 columns]
66
[{'id': 'string', 'pickupInterval': {'start': None, 'finish': None}, 'dropoffInterval': {'start': None, 'finish': None}, 'pickupLocation': {'id': 'string', 'name': 'Ярославль (Ярославская область)', 'latitude': 57.6216145, 'longitude': 39.897878}, 'dropoffLoca

      ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
72  9598  Подтверждён                 44872          44873  20 т / 82 м3   
80  9598  Подтверждён                 44872          44873  20 т / 82 м3   

     Тип кузова                 Груз     Упаковка            Город отгрузки   \
72  Тентованный  Смазочные материалы  Паллета EUR  проспект Губкина, 1, Омск   
80  Тентованный  Смазочные материалы  Паллета EUR  проспект Губкина, 1, Омск   

               Город выгрузки   ...  cost4  date4  route_length4  \
72  проезд Мальцева, 4, Самара  ...    NaN    NaN            NaN   
80  проезд Мальцева, 4, Самара  ...    NaN    NaN            NaN   

    load_capacity4            deliveryCompany  \
72             NaN  Калькулятор по тарифам ЦЛ   
80             NaN                        NaN   

                                    description description1  description2  \
72  Заказ0: нет подходящего маршрута по времени          NaN           NaN   
80                   

      ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
79  2450  Подтверждён                 44762          44767  20 т / 82 м3   

     Тип кузова                 Груз     Упаковка  \
79  Тентованный  Смазочные материалы  Паллета EUR   

                   Город отгрузки  Город выгрузки   ...  cost4  date4  \
79  Ярославская область, Ярославль          Москва  ...    NaN    NaN   

    route_length4  load_capacity4 deliveryCompany  \
79            NaN             NaN             NaN   

                                    description          description1  \
79  Заказ0: нет подходящего маршрута по времени  ТЕСТ: Отдельный А/М.   

    description2  description3 description4  
79           NaN           NaN          NaN  

[1 rows x 52 columns]
79
[{'id': 'string', 'pickupInterval': {'start': None, 'finish': None}, 'dropoffInterval': {'start': None, 'finish': None}, 'pickupLocation': {'id': 'string', 'name': 'Ярославль (Ярославская область)', 'latitude': 57.6216145

      ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
92  4094  Подтверждён                 44788          44790  20 т / 82 м3   

     Тип кузова                 Груз     Упаковка Город отгрузки   \
92  Тентованный  Смазочные материалы  Паллета EUR       Ярославль   

            Город выгрузки   ...  cost4  date4  route_length4  load_capacity4  \
92  Санкт-Петербург, Россия  ...    NaN    NaN            NaN             NaN   

   deliveryCompany                                  description  \
92             NaN  Заказ0: нет подходящего маршрута по времени   

     description1  description2  description3 description4  
92  Отдельный А/М           NaN           NaN          NaN  

[1 rows x 52 columns]
92
[{'id': 'string', 'pickupInterval': {'start': None, 'finish': None}, 'dropoffInterval': {'start': None, 'finish': None}, 'pickupLocation': {'id': 'string', 'name': 'Ярославль (Ярославская область)', 'latitude': 57.6216145, 'longitude': 39.897878}, 'dropoffLocati

      ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
98  1798  Подтверждён                 44750          44755  20 т / 82 м3   

     Тип кузова           Груз      Упаковка  \
98  Тентованный  Металлопрокат  Тарные места   

                                Город отгрузки               Город выгрузки   \
98  Россия, Свердловская область, Верхняя Пышма  Иркутская область, Усть-Кут   

    ...  cost4  date4  route_length4  load_capacity4 deliveryCompany  \
98  ...    NaN    NaN            NaN             NaN             NaN   

                                    description   description1  description2  \
98  Заказ0: нет подходящего маршрута по времени  Отдельный А/М           NaN   

    description3 description4  
98           NaN          NaN  

[1 rows x 52 columns]
98
[{'id': 'string', 'pickupInterval': {'start': None, 'finish': None}, 'dropoffInterval': {'start': None, 'finish': None}, 'pickupLocation': {'id': 'string', 'name': 'Верхняя Пышма (Свердловская 

       ID       Статус  Дата создания заказа  Дата загрузки  Категория ТС  \
104  8456  Подтверждён                 44855          44856  20 т / 82 м3   

      Тип кузова                 Груз     Упаковка  \
104  Тентованный  Смазочные материалы  Паллета EUR   

                                    Город отгрузки   \
104  Московская область, Фрязино, Озёрная улица, 6А   

                         Город выгрузки   ...  cost4  date4  route_length4  \
104  Ростов-на-Дону, улица Туполева, 16А  ...    NaN    NaN            NaN   

     load_capacity4 deliveryCompany  \
104             NaN             NaN   

                                     description description1  description2  \
104  Заказ0: нет подходящего маршрута по времени          NaN           NaN   

     description3 description4  
104           NaN          NaN  

[1 rows x 52 columns]
104
[{'id': 'string', 'pickupInterval': {'start': None, 'finish': None}, 'dropoffInterval': {'start': None, 'finish': None}, 'pickupLocation'

Полученную таблицу экспортируем в эксель.

In [125]:
data_tab.to_excel('output.xlsx')